In [1]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import datetime
from datetime import datetime
import re

### cargar archivo 1

In [7]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padon_1 = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padon_1)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/23336058929/Downloads/mepli/padon1.xlsx' mode='r' encoding='cp1252'>
      Unnamed: 0  Codigo  CUIT_Empleador  CUIL_Titular  Parentesco  \
0              0  127406     20240830214   20240830214           0   
1              1  127406     20119590958   20119590958           0   
2              2  127406     27425694192   27425694192           0   
3              3  127406     27425694192   27425694192           3   
4              4  127406     27270992132   27270992132           0   
...          ...     ...             ...           ...         ...   
1682        1682  127406     20190072577   20190072577           0   
1683        1683  127406     27321459590   27321459590           0   
1684        1684  127406     27174774221   27174774221           0   
1685        1685  127406     27174774221   27174774221           1   
1686        1686  127406     27316755637   27316755637           0   

      CUIL_Familiar Tipo_Doc   Nro_Doc       Apellido  \

### cargar archivo 2

In [ ]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron_2 = pd.read_excel(ruta_archivo, engine='xlrd')
            padron_2 = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron_2)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


### Normalizar DNI archivo convenio

In [3]:

#armar el camp DNI juntando las los campos cuil titular y cuil familiar
padon_1['dni']=padon_1['CUIL_Familiar'].fillna(padon_1['CUIL_Titular'])
#limpiar el nuevo campo dni dejando solo los numeros 
padon_1['dni']=padon_1['dni'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
#dejar solo los numreros eliminar caracteres 
padon_1['dni2']=padon_1['dni'].apply(lambda x: x[2:10] if len(x)>= 8 else '0')
#extraer el dni del cuil y dejar vacios los q son menores  a 8
padon_1['dni2']=padon_1['dni2'].astype(int)
#pasar el campo a int 
padon_1['Fch_Nac']=padon_1['Fch_Nac'].astype(str).str.zfill(8)
#padon_1['Fch_Nac']=padon_1['Fch_Nac'].astype(str)
#convertir fecha. 
print(padon_1['Fch_Nac'])
#padon_1=funciones.transformar_fecha(padon_1, 'Fch_Nac')

#Función para normalizar la fecha
def normalizar_fecha(fecha):
    fecha_normalizada = datetime.strptime(fecha, "%d%m%Y").strftime("%d/%m/%Y")
    return fecha_normalizada

#Aplicar la función a la columna "fech_nacimiento"
padon_1['Fch_Nac'] = padon_1['Fch_Nac'].apply(normalizar_fecha)
padon_1['Fch_Nac'] = padon_1['Fch_Nac'].astype(str)
padon_1[['provincia2','localidad']] = padon_1['Localidad'].str.split('-',expand = True, n=1)
print(padon_1.info(10))


padon_1.to_excel(r"C:\Users\23336058929\Downloads\mepli\padon1.xlsx", engine='openpyxl')

0       01071974
1       25021958
2       20121997
3       12082015
4       01011979
          ...   
1682    17051992
1683    04051986
1684    08011965
1685    03071962
1686    18051985
Name: Fch_Nac, Length: 1687, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687 entries, 0 to 1686
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Codigo             1687 non-null   int64  
 1   CUIT_Empleador     1687 non-null   int64  
 2   CUIL_Titular       1687 non-null   int64  
 3   Parentesco         1687 non-null   int64  
 4   CUIL_Familiar      1687 non-null   int64  
 5   Tipo_Doc           1687 non-null   object 
 6   Nro_Doc            1687 non-null   int64  
 7   Apellido           1687 non-null   object 
 8   Nombre             1687 non-null   object 
 9   Sexo               1687 non-null   object 
 10  Estado_Civil       1687 non-null   int64  
 11  Fch_Nac            1687 non-null  

### Errores

In [6]:


condiciones = [
    lambda row: row['dni2'] is None or len(str(row['dni2'])) < 8,
    lambda row: row['Fch_Nac'] is None or (pd.to_datetime('today') - pd.to_datetime(row['Fch_Nac'], format='%d/%m/%Y')).days // 365.25 > 80,
    lambda row: row['dni2'] >999999 and (pd.to_datetime('today') - pd.to_datetime(row['Fch_Nac'], format='%d/%m/%Y')).days // 365.25 > 25,
    lambda row: row['localidad'] is None
]
mensajes = [
    'familiar mayor de edad',
    'Error en DNI',
    'Error en la edad',
    'localidad vacía'
]

def generar_columnas_error(row):
    errores = {}
    for i in range(len(condiciones)):
        if condiciones[i](row):
            columna_error = f'error{i+1}'
            errores[columna_error] = mensajes[i]
    if errores:
        return pd.Series(errores)
    else:
        return pd.Series()

df_errores = padon_1.apply(generar_columnas_error, axis=1)
print(df_errores.head(20))
df_nuevo = pd.concat([padon_1, df_errores],axis=1)
df_sin_error = df_nuevo[df_errores.any(axis=1)]


#df_sin_error = df_nuevo[df_nuevo['error1'].notna() | df_nuevo['error2'].notna() | df_nuevo['error3'].notna()]

df_sin_error.to_excel(r'C:\Users\23336058929\Downloads\mepli\test_errores.xlsx', engine='openpyxl', index=False)

#df_sin_errores = df_nuevo[df_nuevo['error1'].isnull() & df_nuevo['error2'].isnull() & df_nuevo['error3'].isnull()]

#df_errores.to_excel(r'C:\Users\zickd\Downloads\sin_errores.xlsx', engine='openpyxl', index=False)

#df_sin_errores.to_excel(r'C:\Users\zickd\Downloads\meplife\sin_errores.xlsx', engine='openpyxl', index=False)


C:\Users\23336058929\AppData\Local\Temp\ipykernel_23892\3870773481.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\23336058929\AppData\Local\Temp\ipykernel_23892\3870773481.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\23336058929\AppData\Local\Temp\ipykernel_23892\3870773481.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\23336058929\AppData\Local\Temp\ipykernel_23892\3870773481.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warnin

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 91-05-09 00:00:00

### Merge archivos x DNI

In [ ]:
cruce_tis=pd.merge(padon_1, padron_2, left_on='dni2', right_on='Nro. Documento', how='left', indicator=True)
#cruce_tis.to_excel(r'C:\Users\zickd\Downloads\merge.xlsx', engine='openpyxl', index=False)

cruce_tis_filtro=cruce_tis[cruce_tis['_merge']=='left_only']

#print(cruce_tis_filtro.head(5))

### exportar el cruce

In [ ]:
save_path = filedialog.askdirectory()
#uso iloc por si las columnas tienen el mismo nombre, necesito la primeras 20 columnas
cruce_tis_filtro.iloc[:, :28].to_excel(save_path + '\cruce_meplife.xlsx', engine='openpyxl' ,index = False )



In [7]:
fecha= '01/01/1974'

fecha_correg= (pd.to_datetime('today') - pd.to_datetime(padon_1['Fch_Nac'])).days // 365.25 > 50

edad= (pd.to_datetime('today') - pd.to_datetime(fecha)).days // 365.25

print(fecha_correg)


C:\Users\23336058929\AppData\Local\Temp\ipykernel_21844\2035666616.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  fecha_correg= (pd.to_datetime('today') - pd.to_datetime(padon_1['Fch_Nac'])).days // 365.25 > 50


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 91-09-05 00:00:00 present at position 39